In [1]:
import pandas as pd
import os
from path import Path
import plotly
import plotly.express as px
import plotly.graph_objects as go
import onion_trees as bv
from urllib.request import urlopen
import json
import statsmodels as sm
from statsmodels.formula.api import ols
import mutations as bm
from Bio import Seq, SeqIO, AlignIO, Phylo, Align
from jinja2 import Environment, FileSystemLoader  # html template engine

In [2]:
import reports as br

In [3]:
# cns = SeqIO.parse(gisaid_msa_fp, 'fasta')
# seq_lens = {}
# for rec in cns:
#     length = len(rec.seq)
#     if length != 29903:
#         print(rec.id)
# seq_lens

In [4]:
# output_file = Path('/home/al/analysis/alab_mutations_07-01-2021/sequences_2021-01-06_08-17_cleaned.fasta')
# seqs = SeqIO.parse(gisaid_seqs, 'fasta')
# records = [rec for rec in seqs if rec.id!='England/CAMC-B07F7A/2020']
# SeqIO.write(records, output_file, 'fasta')

In [ ]:
patient_zero = 'hCoV-19/Wuhan/WIV04/2019'
gisaid_seqs = Path('/home/al/analysis/alab_mutations_07-01-2021/sequences_2021-01-06_08-17.fasta')
gisaid_meta = Path('/home/al/analysis/alab_mutations_07-01-2021/metadata_2021-01-06_18-26.tsv')
gisaid_msa_fp = Path('/home/al/analysis/alab_mutations_07-01-2021/msa_0106/msa_0106.fasta')
print("Identifying substitution-based mutations - long...")
gisaid_subs_long, _ = bm.identify_replacements_per_sample(gisaid_msa_fp, 
                                                          gisaid_meta, \
                                                          bm.GENE2POS, 
                                                          data_src='gisaid',
                                                          patient_zero=patient_zero)

Identifying substitution-based mutations - long...
Loading Alignment file at: /home/al/analysis/alab_mutations_07-01-2021/msa_0106/msa_0106.fasta
Initial cleaning...


In [7]:
gisaid_subs_long.shape

NameError: name 'gisaid_subs_long' is not defined

In [ ]:
len(_)

In [6]:
# for rec in cns:
#     if 'hCoV-19/Wuhan/WIV04/2019' in rec.id:
#         print(rec.id)

In [23]:
meta = pd.read_csv(gisaid_meta, sep='\t')
meta.columns

/home/al/anaconda3/envs/covid/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning:

Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.



Index(['strain', 'virus', 'gisaid_epi_isl', 'genbank_accession', 'date',
       'region', 'country', 'division', 'location', 'region_exposure',
       'country_exposure', 'division_exposure', 'segment', 'length', 'host',
       'age', 'sex', 'Nextstrain_clade', 'pangolin_lineage', 'GISAID_clade',
       'originating_lab', 'submitting_lab', 'authors', 'url', 'title',
       'paper_url', 'date_submitted'],
      dtype='object')

In [27]:
meta['strain'].unique().shape

(322744,)

In [33]:
meta.loc[(meta['pangolin_lineage']=='B.1.1.7'), 'strain'].unique().shape

(7435,)

In [32]:
meta.loc[(meta['tmp'].str.len()>=2) & (meta['pangolin_lineage']=='B.1.1.7'), 'strain'].unique().shape

(7360,)

In [34]:
meta.loc[(meta['tmp'].str.len()==3) & (meta['pangolin_lineage']=='B.1.1.7'), 'strain'].unique().shape

(7347,)

In [37]:
gisaid_subs_long['strain'].unique().shape

(202162,)

In [28]:
meta['tmp'] = meta['date'].str.split('-')
meta[(meta['tmp'].str.len()>=2)]['strain'].unique().shape

(319385,)

In [20]:
gisaid_data['idx'].unique().shape

(202162,)

In [20]:
# gisaid_data['pangolin_lineage'].unique()

In [7]:
feature = 'pangolin_lineage'
values = ['B.1.1.7']
meta_fp = Path('/home/al/code/HCoV-19-Genomics/metadata.csv')
tree_fp = Path('/home/al/analysis/alab_mutations_01-01-2021/alab/seqs_aligned.fa.treefile')
subs_fp = '/home/al/analysis/alab_mutations_01-01-2021/alab_substitutions_long_01-01-2021.csv'
country_fp = '/home/al/data/geojsons/countries.geo.json'
states_fp = "/home/al/data/geojsons/us-states.json"
patient_zero = 'NC_045512.2'
gisaid_data = gisaid_subs_long[gisaid_subs_long['host']=='Human']

In [8]:
fd, gdp, aadp, saadp, wmap, smap = br.generate_voc_data(feature, values, gisaid_data, tree_fp, 
                                                        subs_fp, meta_fp, states_fp, 
                                                        patient_zero)

/home/al/anaconda3/envs/covid/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning:

Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.



In [13]:
html = generate_voc_html(feature, values, fd, wmap, smap, gdp, aadp, saadp)
br.save_html(html, 'voc_test6_b117.html')

In [180]:
gisaid[gisaid['pangolin_lineage']=='B.1.1.7']['date'].min()

Timestamp('2020-01-01 00:00:00')

In [165]:
def generate_html(feature, values, first_detected, world_map, state_map, genetic_distance_plot,
                  aa_distance_plot, s_aa_distance_plot):
    # express plots in html and JS
    world_map = plotly.offline.plot(world_map, include_plotlyjs=False, output_type='div')
    state_map = plotly.offline.plot(state_map, include_plotlyjs=False, output_type='div')
#     county_map = plotly.offline.plot(county_map, include_plotlyjs=False, output_type='div')
    genetic_distance_plot = plotly.offline.plot(genetic_distance_plot, include_plotlyjs=False, output_type='div')
    aa_distance_plot = plotly.offline.plot(aa_distance_plot, include_plotlyjs=False, output_type='div')
    s_aa_distance_plot = plotly.offline.plot(s_aa_distance_plot, include_plotlyjs=False, output_type='div')
    # generate output messages
    #TODO: expt_name, first_detected
    # dir containing our template
    file_loader = FileSystemLoader('templates')
    # load the environment
    env = Environment(loader=file_loader)
    # load the template
    template = env.get_template('voc.html')
    # render data in our template format
    html_output = template.render(feature=feature, values=values,
                                  world_map=world_map, state_map=state_map, 
                                  genetic_distance_plot=genetic_distance_plot, 
                                  aa_distance_plot=aa_distance_plot, s_aa_distance_plot=s_aa_distance_plot,
                                  first_detected=first_detected)
    return html_output


def save_html(html_output: str, filename: str):
    with open(filename, 'w') as f:
        f.write(html_output)

In [166]:
def generate_data(feature, values, gisaid_data, tree_fp, subs_fp,
                  meta_fp, states_fp, patient_zero):
    genetic_distance_plot = genetic_distance(tree_fp, meta_fp, patient_zero)
    aa_distance_plot = aa_distance(subs_fp, meta_fp)
    s_aa_distance_plot = s_aa_distance(subs_fp, meta_fp)
    state_map, _, _ = map_by_state(gisaid_data, feature, values, states_fp)
    world_map, _, _ = map_by_country(gisaid_data, feature, values)
    r = gisaid.loc[gisaid[feature].isin(values)]
    date = r['date_submitted'].min()
    state = r[r['date_submitted']==date]['division'].unique()
    cntry = r[r['date_submitted']==date]['country'].unique()
    first_detected = f"The {values} {feature} was first detected on {date} in {state}, {cntry}"
    return first_detected, genetic_distance_plot, aa_distance_plot, s_aa_distance_plot, world_map, state_map

In [137]:
alab_subs.columns

Index(['idx', 'sequence', 'replacements', 'pos', 'gene', 'codon_num',
       'ref_codon', 'alt_codon', 'ref_aa', 'alt_aa', 'ID', 'gb_accession',
       'gisaid_accession', 'collection_date', 'location',
       'percent_coverage_cds', 'avg_depth', 'authors', 'originating_lab',
       'fasta_hdr', 'date', 'nonsyn', 'S_nonsyn'],
      dtype='object')

In [ ]:
alab_subs = pd.read_csv('/home/al/analysis/alab_mutations_01-01-2021/alab_substitutions_long_01-01-2021.csv')

In [141]:
alab_subs['nonsyn'] = False
alab_subs.loc[alab_subs['ref_aa']!=alab_subs['alt_aa'], 'nonsyn'] = True
alab_subs['S_nonsyn'] = False
alab_subs.loc[(alab_subs['gene']=='S') & (alab_subs['ref_aa']!=alab_subs['alt_aa']), 'S_nonsyn'] = True

In [143]:
alab_subs.groupby('idx').agg(num_nonsyn_muts=('nonsyn', 'sum'), num_S_nonsyn_muts=('S_nonsyn', 'sum')).reset_index()

,idx,num_nonsyn_muts,num_S_nonsyn_muts
0,Consensus_MG0987,4.0,1.0
1,Consensus_PC00101P_threshold_0_quality_20,2.0,1.0
2,Consensus_SEARCH-0007-SAN_L1_threshold_0_quali...,6.0,1.0
3,Consensus_SEARCH-0016-SAN_L1_threshold_0_quali...,2.0,0.0
4,Consensus_SEARCH-0017-SAN_L1_threshold_0_quali...,6.0,1.0
...,...,...,...
3437,hCoV-19/USA/SEARCH-5566-SAN/2020,13.0,1.0
3438,hCoV-19/USA/SEARCH-5567-SAN/2020,12.0,2.0
3439,hCoV-19/USA/SEARCH-5570-SAN/2020,14.0,1.0
3440,hCoV-19/USA/SEARCH-5573-SAN/2020,10.0,3.0


In [153]:
def aa_distance(subs_fp, meta_fp, alpha=0.05):
    alab_subs = pd.read_csv(subs_fp)
    alab_subs['nonsyn'] = False
    alab_subs.loc[alab_subs['ref_aa']!=alab_subs['alt_aa'], 'nonsyn'] = True
    alab_subs['S_nonsyn'] = False
    alab_subs.loc[(alab_subs['gene']=='S') & (alab_subs['ref_aa']!=alab_subs['alt_aa']), 'S_nonsyn'] = True
    dists_df = (alab_subs.groupby('fasta_hdr')
                .agg(num_nonsyn_muts=('nonsyn', 'sum'), num_S_nonsyn_muts=('S_nonsyn', 'sum'))
                .reset_index())
    meta = pd.read_csv(meta_fp)
    sd_meta = meta[meta['location'].str.contains('San Diego')]
    df = pd.merge(dists_df, sd_meta, on='fasta_hdr')
    df['date'] = pd.to_datetime(df['collection_date'], errors='coerce')
    df['month'] = df['date'].dt.month
    df['doy'] = df['date'].dt.dayofyear
    df = df.loc[~df['doy'].isna()]
    model = ols('num_nonsyn_muts ~ doy', data=df).fit()
    df['predict'] = model.predict(df['doy'])
    df['p'] = model.outlier_test(method='fdr_bh')['fdr_bh(p)']
    df['outlier'] = False
    df.loc[df['p']<alpha, 'outlier'] = True
    fig = go.Figure(data=go.Scatter(y=df[df['outlier']==False]['num_nonsyn_muts'], x=df[df['outlier']==False]['doy'], 
                                name='samples', mode='markers', marker_color='rgba(30,144,255,.6)'))
    fig.add_trace(go.Scatter(y=df[df['outlier']==True]['num_nonsyn_muts'], x=df[df['outlier']==True]['doy'],
                             mode='markers', marker_color='rgba(220,20,60,.6)', name='SoIs',
                 text=df[df['outlier']==True][['ID', 'date']],
                 hovertemplate = 
                 "<b>%{text[0]}</b><br>" +
                 "<b>%{text[1]}</b><br>"))
    fig.add_trace(go.Scatter(y=df['predict'], x=df['doy'], name='OLS', mode='lines', line_color='rgba(0,0,0,1.)'))
    fig.update_layout(yaxis_title='Amino Acid Changes (root-to-tip)', xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850, width=800)
    return fig

In [149]:
meta_fp

Path('/home/al/code/HCoV-19-Genomics/metadata.csv')

In [21]:
# fig = aa_distance(subs_fp, meta_fp)
# fig.show()

In [22]:
def s_aa_distance(subs_fp, meta_fp, alpha=0.05):
    alab_subs = pd.read_csv(subs_fp)
    alab_subs['nonsyn'] = False
    alab_subs.loc[alab_subs['ref_aa']!=alab_subs['alt_aa'], 'nonsyn'] = True
    alab_subs['S_nonsyn'] = False
    alab_subs.loc[(alab_subs['gene']=='S') & (alab_subs['ref_aa']!=alab_subs['alt_aa']), 'S_nonsyn'] = True
    dists_df = (alab_subs.groupby('fasta_hdr')
                .agg(num_nonsyn_muts=('nonsyn', 'sum'), num_S_nonsyn_muts=('S_nonsyn', 'sum'))
                .reset_index())
    meta = pd.read_csv(meta_fp)
    sd_meta = meta[meta['location'].str.contains('San Diego')]
    df = pd.merge(dists_df, sd_meta, on='fasta_hdr')
    df['date'] = pd.to_datetime(df['collection_date'], errors='coerce')
    df['month'] = df['date'].dt.month
    df['doy'] = df['date'].dt.dayofyear
    df = df.loc[~df['doy'].isna()]
    model = ols('num_S_nonsyn_muts ~ doy', data=df).fit()
    df['predict'] = model.predict(df['doy'])
    df['p'] = model.outlier_test(method='fdr_bh')['fdr_bh(p)']
    df['outlier'] = False
    df.loc[df['p']<alpha, 'outlier'] = True
    fig = go.Figure(data=go.Scatter(y=df[df['outlier']==False]['num_S_nonsyn_muts'], x=df[df['outlier']==False]['doy'], 
                                name='samples', mode='markers', marker_color='rgba(30,144,255,.6)'))
    fig.add_trace(go.Scatter(y=df[df['outlier']==True]['num_S_nonsyn_muts'], x=df[df['outlier']==True]['doy'],
                             mode='markers', marker_color='rgba(220,20,60,.6)', name='SoIs',
                 text=df[df['outlier']==True][['ID', 'date']],
                 hovertemplate = 
                 "<b>%{text[0]}</b><br>" +
                 "<b>%{text[1]}</b><br>"))
    fig.add_trace(go.Scatter(y=df['predict'], x=df['doy'], name='OLS', mode='lines', line_color='rgba(0,0,0,1.)'))
    fig.update_layout(yaxis_title='Amino Acid Changes in the S protein(root-to-tip)', xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850, width=800)
    return fig

# fig = s_aa_distance(subs_fp, meta_fp)
# fig.show()

In [39]:
def genetic_distance(tree_fp, meta_fp, patient_zero, alpha=0.05):
    tree = bv.load_tree(tree_fp, patient_zero)
    dists = {n.name: tree.distance(n.name, patient_zero) for n in tree.get_terminals()}
    dists_df = (pd.DataFrame(index=dists.keys(), data=dists.values(), 
                      columns=['genetic_distance'])
         .reset_index()
         .rename(columns={'index': 'fasta_hdr'}))
    meta = pd.read_csv(meta_fp)
    sd_meta = meta[meta['location'].str.contains('San Diego')]
    df = pd.merge(dists_df, sd_meta, on='fasta_hdr')
    df['date'] = pd.to_datetime(df['collection_date'], errors='coerce')
    df['month'] = df['date'].dt.month
    df['doy'] = df['date'].dt.dayofyear
    df = df.loc[~df['doy'].isna()]
    model = ols('genetic_distance ~ doy', data=df).fit()
    df['predict'] = model.predict(df['doy'])
    df['p'] = model.outlier_test(method='fdr_bh')['fdr_bh(p)']
    df['outlier'] = False
    df.loc[df['p']<alpha, 'outlier'] = True
    fig = go.Figure(data=go.Scatter(y=df[df['outlier']==False]['genetic_distance'], x=df[df['outlier']==False]['doy'], 
                                name='samples', mode='markers', marker_color='rgba(30,144,255,.6)'))
    fig.add_trace(go.Scatter(y=df[df['outlier']==True]['genetic_distance'], x=df[df['outlier']==True]['doy'],
                             mode='markers', marker_color='rgba(220,20,60,.6)', name='SoIs',
                 text=df[df['outlier']==True][['ID', 'date']],
                 hovertemplate = 
                 "<b>%{text[0]}</b><br>" +
                 "<b>%{text[1]}</b><br>"))
    fig.add_trace(go.Scatter(y=df['predict'], x=df['doy'], name='OLS', mode='lines', line_color='rgba(0,0,0,1.)'))
    fig.update_layout(yaxis_title='Genetic Distance (root-to-tip)', xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850, width=800)
    return fig

In [23]:
def map_by_state(data: pd.DataFrame, feature: str, values: list, states_fp: str):
    with open(states_fp) as f:
        states = json.load(f)
    state_map = {x['properties']['name']: x['id'] for x in states['features']}
    results = data.loc[(data[feature].isin(values)) & (data['country']=='United States of America')]
    results_by_state = results.groupby('division').agg(num_samples=('idx', 'nunique')).reset_index()
    results_by_state['id'] = results_by_state['division'].apply(lambda x: state_map.get(x, 'unk'))
#     fig = px.choropleth(results_by_state, geojson=states, scope="usa",
#                                locations='id', color='num_samples',# locationmode='USA-states',
#                                color_continuous_scale="bluered",
#                                range_color=(0, results_by_state['num_samples'].max()),
# #                                labels={'num_samples': f'Number of samples with {values}: ', 'division': 'loc:'},
#                                hover_data=['division', 'num_samples']
#                               )
    fig = px.choropleth_mapbox(results_by_state, geojson=states, 
                               locations='id', color='num_samples',
                               color_continuous_scale="bluered", center={"lat": 37.0902, "lon": -95.7129},
                               range_color=(0, results_by_state['num_samples'].max()),
                               mapbox_style="carto-positron", zoom=3,
                               opacity=0.5,
                               hover_data=['division', 'num_samples']
                               #labels={'num_samples':f'Number of samples with {values}', 'division': f'location:'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig, state_map, results_by_state
# states_fp = "/home/al/code/MappingAPI/data/geojson/us-states.json"
# fig, mapp, df = map_by_state(gisaid, 'mutation', ['S:501Y'], states_fp)
# fig.show()

In [24]:
def map_by_country(data: pd.DataFrame, feature: str, values: list):
    with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
        countries = json.load(response)
    for c in countries['features']:
        if c['id']=='USA':
            assert c['properties']['name'] == 'United States of America'
    country_map = {x['properties']['name']: x['id'] for x in countries['features']}
    results = data.loc[data[feature].isin(values)]
    results_by_cntry = results.groupby('country').agg(num_samples=('idx', 'nunique')).reset_index()
    results_by_cntry['id'] = results_by_cntry['country'].apply(lambda x: country_map.get(x, 'unk'))
    fig = px.choropleth_mapbox(results_by_cntry, geojson=countries, 
                               locations='id', color='num_samples',
                               color_continuous_scale="bluered",
                               range_color=(0, results_by_cntry['num_samples'].max()),
                               mapbox_style="carto-positron", zoom=1,
                               opacity=0.5,
                               labels={'num_samples':f'Number of samples with {values}', 'id': f'location:'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig, country_map, results_by_cntry

# fig, mapp, df = map_by_country(gisaid, 'mutation', ['S:501Y'])
# fig.show()

In [2]:
meta_fp = Path('/home/al/code/HCoV-19-Genomics/metadata.csv')
tree_fp = Path('/home/al/analysis/alab_mutations_01-01-2021/alab/seqs_aligned.fa.treefile')
patient_zero = 'NC_045512.2'
tree = bv.load_tree(tree_fp, patient_zero)

In [3]:
dists = {n.name: tree.distance(n.name, patient_zero) for n in tree.get_terminals()}

In [4]:
dists_df = (pd.DataFrame(index=dists.keys(), data=dists.values(), 
                      columns=['genetic_distance'])
         .reset_index()
         .rename(columns={'index': 'fasta_hdr'}))
dists_df.head()

,fasta_hdr,genetic_distance
0,Consensus_MG0987,0.000204
1,Consensus_SEARCH-0007-SAN_L1_threshold_0_quali...,0.000335
2,hCoV-19/USA/SEARCH-0438-SAN/2020,0.000303
3,hCoV-19/USA/ALSR-1460/2020,0.000236
4,Consensus_SEARCH-0032-JOR_L1_L2_L3_L4_threshol...,0.000404


In [5]:
meta = pd.read_csv(meta_fp)
meta.head()

,ID,gb_accession,gisaid_accession,collection_date,location,percent_coverage_cds,avg_depth,authors,originating_lab,fasta_hdr
0,MG0987,MT598172,EPI_ISL_416457,2020-03-18,USA/California/San Diego,99.5954,2465.60,SEARCH Alliance San Diego,Andersen lab at Scripps Research,Consensus_MG0987
1,PC00101P,MT192765,EPI_ISL_414648,2020-03-11,USA/California/San Diego,99.7525,3516.14,SEARCH Alliance San Diego,Andersen lab at Scripps Research,Consensus_PC00101P_threshold_0_quality_20
2,SEARCH-0007-SAN,MT598171,EPI_ISL_429990,2020-03-21,USA/California/San Diego,100.0000,6215.17,SEARCH Alliance San Diego with Christina Clark...,Rady's Childrens Hospital,Consensus_SEARCH-0007-SAN_L1_threshold_0_quali...
3,SEARCH-0016-SAN,MT598173,EPI_ISL_430016,2020-03-24,USA/California/San Diego,100.0000,6440.67,SEARCH Alliance San Diego,Andersen lab at Scripps Research,Consensus_SEARCH-0016-SAN_L1_threshold_0_quali...
4,SEARCH-0017-SAN,MT598174,EPI_ISL_429991,2020-03-24,USA/California/San Diego,100.0000,4947.09,SEARCH Alliance San Diego,Andersen lab at Scripps Research,Consensus_SEARCH-0017-SAN_L1_threshold_0_quali...


In [6]:
print(meta.shape)
sd_meta = meta[meta['location'].str.contains('San Diego')]
print(sd_meta.shape)

(3447, 10)
(2399, 10)


In [7]:
print(dists_df.shape)
df = pd.merge(dists_df, sd_meta, on='fasta_hdr')
print(df.shape)

(3448, 2)
(2398, 11)


In [8]:
df['date'] = pd.to_datetime(df['collection_date'], errors='coerce')

In [9]:
df['month'] = df['date'].dt.month

In [10]:
# df

In [11]:
df['doy'] = df['date'].dt.dayofyear

In [12]:
df = df.loc[~df['doy'].isna()]

In [13]:
model = ols('genetic_distance ~ doy', data=df).fit()

In [14]:
df['predict'] = model.predict(df['doy'])

In [15]:
alpha = 0.05

In [16]:
df['p'] = model.outlier_test(method='fdr_bh')['fdr_bh(p)']
df['outlier'] = False
df.loc[df['p']<alpha, 'outlier'] = True

In [25]:
# fig = go.Figure(data=go.Scatter(y=df[df['outlier']==False]['genetic_distance'], x=df[df['outlier']==False]['doy'], 
#                                 name='samples', mode='markers', marker_color='rgba(30,144,255,.6)'))
# fig.add_trace(go.Scatter(y=df[df['outlier']==True]['genetic_distance'], x=df[df['outlier']==True]['doy'],
#                          mode='markers', marker_color='rgba(220,20,60,.6)', name='SoIs',
#              text=df[df['outlier']==True][['ID', 'date']],
#              hovertemplate = 
#              "<b>%{text[0]}</b><br>" +
#              "<b>%{text[1]}</b><br>"))
# fig.add_trace(go.Scatter(y=df['predict'], x=df['doy'], name='OLS', mode='lines', line_color='rgba(0,0,0,1.)'))
# fig.update_layout(yaxis_title='Genetic Distance (root-to-tip)', xaxis_title='Collection Date',
#                   template='plotly_white', autosize=True)#, height=850, width=800)
# fig.show()

Samples are designated as Sample of Interest (SoI) based on the Benjamini-Hochberg outlier test at a 5% significance level. 

In [26]:
# fig = go.Figure(go.Box(y=df['genetic_distance'], x=df['month'], text=df[['ID', 'date']],
#             hovertemplate = 
#             "<b>%{text[0]}</b><br>" +
#             "<b>%{text[1]}</b><br>"))
# fig.update_layout(yaxis_title='Genetic Distance', xaxis_title='Collection Month',
#                   template='plotly_white', autosize=False, height=850, width=800)
# fig.show()

# Strain Prevalence Maps

## Country Level

In [19]:
with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
    countries = json.load(response)
for c in countries['features']:
    if c['id']=='USA':
        assert c['properties']['name'] == 'United States of America'

In [21]:
gisaid_seqs = Path('/home/al/analysis/gisaid/sequences_2021-01-01_08-20.fasta')
gisaid_meta = Path('/home/al/analysis/gisaid/metadata_2021-01-01_08-12.tsv')
gisaid_msa_fp = Path(gisaid_seqs.split('.')[0] + '_aligned.fa')
print("Identifying substitution-based mutations - long...")
gisaid_subs_long, _ = bm.identify_replacements_per_sample(gisaid_msa_fp, gisaid_meta, bm.GENE2POS, data_src='gisaid')

Identifying substitution-based mutations - long...
Loading Alignment file at: /home/al/analysis/gisaid/sequences_2021-01-01_08-20_aligned.fa
Initial cleaning...
Identifying mutations...
Mapping Genes to mutations...
Compute codon numbers...
Fetch reference codon...
Fetch alternative codon...
Map amino acids...
Fuse with metadata...


/home/al/anaconda3/envs/covid/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning:

Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.



In [22]:
# gisaid = pd.read_csv('/home/al/analysis/alab_mutations_01-01-2021/gisaid_substitutions_long_01-01-2021.csv')

In [27]:
gisaid = gisaid_subs_long
gisaid['mutation'] = gisaid['gene'] + ':' + gisaid['codon_num'].astype(str) + gisaid['alt_aa']
gisaid.loc[gisaid['country']=='USA', 'country'] = 'United States of America'

In [28]:
country_map = {x['properties']['name']: x['id'] for x in countries['features']}

In [29]:
gisaid.loc[gisaid['pangolin_lineage']=='B.1.1.7'].drop_duplicates(subset=['idx'])['country'].value_counts()

United Kingdom              4607
Denmark                       33
Portugal                      20
Italy                         17
Netherlands                    8
Ireland                        7
Singapore                      6
Japan                          6
Australia                      5
Canada                         5
Israel                         4
India                          3
South Korea                    3
Brazil                         2
Hong Kong                      2
Finland                        2
Germany                        2
France                         2
United States of America       2
Norway                         2
Switzerland                    1
Spain                          1
Sweden                         1
Name: country, dtype: int64

In [30]:
gisaid['id'] = gisaid['country'].apply(lambda x: country_map.get(x, 'unk'))
gisaid['id'].unique()

array(['CHN', 'USA', 'TWN', 'FRA', 'AUS', 'DEU', 'unk', 'GBR', 'JPN',
       'KOR', 'BEL', 'VNM', 'NPL', 'ITA', 'KHM', 'SWE', 'BRA', 'CAN',
       'FIN', 'MEX', 'CHE', 'NZL', 'IND', 'NGA', 'NLD', 'LUX', 'PRT',
       'CZE', 'ESP', 'CHL', 'IRL', 'COD', 'PAN', 'GEO', 'DNK', 'RUS',
       'PER', 'HUN', 'SAU', 'KWT', 'POL', 'LTU', 'MYS', 'ZAF', 'TUR',
       'PAK', 'ISL', 'ECU', 'NOR', 'SVK', 'COL', 'SEN', 'DZA', 'GRC',
       'ISR', 'AUT', 'LVA', 'BLR', 'EST', 'SVN', 'ARG', 'GHA', 'THA',
       'IRN', 'URY', 'LKA', 'GMB', 'UKR', 'HRV', 'JOR', 'PHL', 'EGY',
       'CRI', 'MMR', 'KAZ', 'ARE', 'IDN', 'BRN', 'BGD', 'ROU', 'LBN',
       'JAM', 'UGA', 'MAR', 'OMN', 'KEN', 'TUN', 'BIH', 'CYP', 'MNG',
       'VEN', 'BEN', 'BGR', 'MLI', 'MDG', 'GTM', 'BLZ', 'ZMB', 'SLE',
       'CMR', 'CUB', 'MNE', 'MDA', 'SUR', 'DOM', 'GAB', 'CS-KM', 'BWA',
       'MLT', 'COG', 'IRQ', 'RWA', 'MOZ', 'ZWE', 'GNQ', 'BFA', 'SLV',
       'ARM', 'PNG'], dtype=object)

In [31]:
# gisaid.loc[(gisaid['pangolin_lineage']=='B.1.1.7') & (gisaid['country']=='Jordan')]

In [27]:
def map_by_country(data: pd.DataFrame, feature: str, values: list, countries_geojson):
    country_map = {x['properties']['name']: x['id'] for x in countries['features']}
    results = data.loc[data[feature].isin(values)]
    results_by_cntry = results.groupby('country').agg(num_samples=('idx', 'nunique')).reset_index()
    results_by_cntry['id'] = results_by_cntry['country'].apply(lambda x: country_map.get(x, 'unk'))
    fig = px.choropleth_mapbox(results_by_cntry, geojson=countries_geojson, 
                               locations='id', color='num_samples',
                               color_continuous_scale="bluered",
                               range_color=(0, results_by_cntry['num_samples'].max()),
                               mapbox_style="carto-positron", zoom=1,
                               opacity=0.5,
                               labels={'num_samples':f'Number of samples with {values}', 'id': f'location:'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig, country_map, results_by_cntry

# fig, mapp, df = map_by_country(gisaid, 'mutation', ['S:501Y'], countries)
# fig.show()

In [27]:
with open(states_fp) as f:
    states = json.load(f)

In [28]:
# states['features'][0]
# states

In [29]:
state_map = {x['properties']['name']: x['id'] for x in states['features']}

In [30]:
# state_map

In [31]:
# gisaid['division']

In [32]:
gisaid['id'] = gisaid['division'].apply(lambda x: state_map.get(x, 'unk'))
gisaid['id'].unique()

array(['unk', '53', '06', '04', '55', '25', '17', '48', '36', '27', '49',
       '09', '41', '51', '44', '13', '12', '02', '33', '56', '18', '34',
       '39', '19', '16', '37', '45', '20', '22', '24', '29', '31', '32',
       '42', '26', '08', '15', '72', '30', '50', '01', '05', '35', '47',
       '21', '40', '28', '46', '23', '54', '10'], dtype=object)

In [33]:
results = gisaid.loc[gisaid['mutation']=='S:501Y']
results_by_state = results.groupby('division').agg(num_samples=('idx', 'nunique')).reset_index()
results_by_state['division'].apply(lambda x: state_map.get(x, 'unk'))

0     unk
1     unk
2     unk
3     unk
4      17
5     unk
6      25
7     unk
8      36
9     unk
10     41
11    unk
12    unk
13    unk
14    unk
15    unk
Name: division, dtype: object

In [38]:
df

,division,num_samples,id
0,Illinois,1,17
1,Massachusetts,5,25
2,New York,1,36
3,Oregon,1,41


In [28]:
def map_by_state(data: pd.DataFrame, feature: str, values: list, states_fp: str):
    with open(states_fp) as f:
        states = json.load(f)
    state_map = {x['properties']['name']: x['id'] for x in states['features']}
    results = data.loc[(data[feature].isin(values)) & (data['country']=='United States of America')]
    results_by_state = results.groupby('division').agg(num_samples=('idx', 'nunique')).reset_index()
    results_by_state['id'] = results_by_state['division'].apply(lambda x: state_map.get(x, 'unk'))
    fig = px.choropleth(results_by_state, geojson=states, scope="usa",
                               locations='id', color='num_samples',
                               color_continuous_scale="bluered",
                               range_color=(0, results_by_state['num_samples'].max()),
                               labels={'num_samples': f'Number of samples with {values}: ', 'division': 'loc:'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig, state_map, results_by_state
# states_fp = "/home/al/code/MappingAPI/data/geojson/us-states.json"
# fig, mapp, df = map_by_state(gisaid, 'mutation', ['S:501Y'], states_fp)
# fig.show()

# Report Generation

In [ ]:
def generate_html(world_map, state_map, county_map, genetic_distance_plot):
    # express plots in html and JS
    world_map = plotly.offline.plot(world_map, include_plotlyjs=False, output_type='div')
    state_map = plotly.offline.plot(state_map, include_plotlyjs=False, output_type='div')
    county_map = plotly.offline.plot(county_map, include_plotlyjs=False, output_type='div')
    genetic_distance_plot = plotly.offline.plot(genetic_distance_plot, include_plotlyjs=False, output_type='div')
    # generate output messages
    #TODO: expt_name, first_detected
    # dir containing our template
    file_loader = FileSystemLoader('templates')
    # load the environment
    env = Environment(loader=file_loader)
    # load the template
    template = env.get_template('voc.html')
    # render data in our template format
    html_output = template.render(world_map=world_map, state_map=state_map, 
                                  county_map=county_map, genetic_distance_plot=genetic_distance_plot,
                                 first_detected=first_detected, expt_name=expt_name)
    return html_output


def save_html(html_output: str, filename: str):
    with open(filename, 'w') as f:
        f.write(html_output)

In [ ]:
df = pd.read_csv()

In [4]:
out_dir = Path('/home/al/analysis/mutations/gisaid')

In [316]:
gisaid = pd.read_csv(out_dir/'gisaid_replacements_19-12-2020.csv')

In [317]:
# gisaid['mutation'] = gisaid['gene'] + ':' + gisaid['codon_num'].astype(str) + gisaid['alt_aa']
gisaid.loc[gisaid['country']=='USA', 'country'] = 'United States of America'

In [304]:
# gisaid.to_csv(out_dir/'gisaid_replacements_19-12-2020.csv', index=False)

In [324]:
# gisaid['country'].unique()

In [206]:
## Analyzing A-lab's submission rates across the U.S.
us = gisaid[gisaid['country']=='USA'].copy()
us.loc[:, 'date'] = pd.to_datetime(us['date'])
us.loc[:, 'month'] = us['date'].dt.month
us.drop_duplicates(subset=['idx'], inplace=True)
us.loc[:, 'is_andersen'] = False
us.loc[us['submitting_lab'].str.contains('Andersen'), 'is_andersen'] = True
ans = (us.groupby(['submitting_lab'])
 .agg(num_samples=('idx', 'nunique'))
 .reset_index()
 .sort_values(['num_samples'], ascending=[False])
 .reset_index())
# ans[ans['month']==11]
ans.iloc[:10]

,index,submitting_lab,num_samples
0,48,Houston Methodist Hospital,5069
1,137,Utah Public Health Laboratory,2977
2,115,Seattle Flu Study,2869
3,16,Chan-Zuckerberg Biohub,2810
4,111,Quest Diagnostics,2708
5,126,UW Virology Lab,2645
6,81,Michigan Department of Health and Human Servic...,2520
7,119,TGen North,2511
8,4,Andersen lab at Scripps Research,2307
9,136,University of Wisconsin-Madison AIDS Vaccine R...,2226


In [ ]:
uk_seqs = uk.groupby(['date', 'idx']).agg(num_nonsyn_mutations=('is_nonsyn_mutation', 'sum'),
                                num_S_nonsyn_mutations=('is_S_nonsyn_mutation', 'sum')).reset_index()
uk_seqs.loc[:, 's501y'] = False
uk_seqs.loc[uk_seqs['idx'].isin(mutant_samples), 's501y'] = True

In [158]:
s501y_filter = (gisaid['gene']=='S') & (gisaid['codon_num']==501) & (gisaid['alt_aa']=='Y')
gisaid.loc[(s501y_filter), 'country'].value_counts()

United Kingdom    2160
South Africa       201
Australia           39
Denmark             10
USA                  8
Brazil               1
Name: country, dtype: int64

In [156]:
b117_samples = gisaid.loc[(gisaid['pangolin_lineage']=='B.1.1.7'), 'country'].unique()
b117_samples

array(['United Kingdom', 'Iceland', 'France', 'Czech Republic',
       'United Arab Emirates', 'Australia', 'New Zealand', 'Jordan',
       'Denmark'], dtype=object)

In [115]:
uk = gisaid[gisaid['country']=='United Kingdom']

In [125]:
# s501y_filter = (uk['gene']=='S') & (uk['codon_num']==501) & (uk['alt_aa']=='Y')
s501y_filter = (uk['pangolin_lineage']=='B.1.1.7')
uk.loc[s501y_filter]['date'].min()

'2020-10-05'

In [126]:
# filtering out UK samples AFTER S501Y was first detected
uk = uk.loc[uk['date'] > uk.loc[s501y_filter]['date'].min()]

In [127]:
# uk.columns

In [153]:
uk['date'].min()

Timestamp('2020-10-07 00:00:00')

In [129]:
all_samples = uk['idx'].unique()
len(all_samples)

67582

In [130]:
mutant_samples = uk.loc[s501y_filter, 'idx'].unique()
len(mutant_samples)

1753

In [131]:
# common = set(mutant_samples) & set(b117_samples)
# len(common)

In [133]:
uk.loc[:, 's501y'] = False
uk.loc[s501y_filter, 's501y'] = True

In [134]:
uk.loc[:, 'is_nonsyn_mutation'] = False
uk.loc[uk['alt_aa']!=uk['ref_aa'], 'is_nonsyn_mutation'] = True
uk.loc[:, 'is_S_nonsyn_mutation'] = False
uk.loc[(uk['alt_aa']!=uk['ref_aa']) & (uk['gene']=='S'), 'is_S_nonsyn_mutation'] = True

In [136]:
uk['date'] = pd.to_datetime(uk['date'])

<ipython-input-136-9ebf2725a3c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk['date'] = pd.to_datetime(uk['date'])


In [151]:
uk[['is_nonsyn_mutation', 'is_S_nonsyn_mutation']].sum() / len(all_samples)

is_nonsyn_mutation      10.325974
is_S_nonsyn_mutation     2.629310
dtype: float64

In [149]:
uk_seqs = uk.groupby(['date', 'idx']).agg(num_nonsyn_mutations=('is_nonsyn_mutation', 'sum'),
                                num_S_nonsyn_mutations=('is_S_nonsyn_mutation', 'sum')).reset_index()
uk_seqs.loc[:, 's501y'] = False
uk_seqs.loc[uk_seqs['idx'].isin(mutant_samples), 's501y'] = True

In [150]:
uk_seqs.loc[:, 'year'] = uk_seqs['date'].dt.year
uk_seqs.loc[:, 'month'] = uk_seqs['date'].dt.month
uk_seqs.loc[:, 'week'] = uk_seqs['date'].dt.isocalendar().week
mnthly_cnts = uk_seqs.groupby(['year', 'month', 'week']).agg(total_samples=('idx', 'nunique'), 
                                                             mutated_samples=('s501y', 'sum'))
mnthly_cnts['mutation_freq'] = mnthly_cnts['mutated_samples'] / mnthly_cnts['total_samples']
mnthly_cnts

total_samples  mutated_samples  mutation_freq
year month week                                               
2020 10    41              760                0       0.000000
           42             2643                0       0.000000
           43             6290                1       0.000159
           44             6892                3       0.000435
     11    45             9493                6       0.000632
           46             5970               10       0.001675
           47             7690               82       0.010663
           48             8867              212       0.023909
           49             4418               96       0.021729
     12    49             4341              358       0.082469
           50             3264              146       0.044730
           51             6954              839       0.120650

In [223]:
x = uk_seqs[uk_seqs['s501y']==True].sample(1000)
y = uk_seqs[uk_seqs['s501y']==False].sample(1000)
sample_data = pd.concat([x, y])

In [224]:
sample_data.groupby('s501y').agg({'num_nonsyn_mutations': ['mean', 'median'], 'num_S_nonsyn_mutations': ['mean', 'median']})

num_nonsyn_mutations        num_S_nonsyn_mutations       
                      mean median                   mean median
s501y                                                          
False                9.998      9                  2.473      2
True                25.917     26                  7.115      7

In [225]:
# box plots of non-synonymous mutations

In [4]:
fig = go.Figure(go.Box(y=sample_data['num_nonsyn_mutations'], x=sample_data['s501y'], boxpoints=False))
fig.update_layout(yaxis_title='Non-synonymous Mutations', xaxis_title='B1117 Lineage',
                  template='plotly_white', autosize=False, height=850, width=800)
# fig.show()

NameError: name 'sample_data' is not defined

In [ ]:
fig = go.Figure(go.Box(y=sample_data['num_S_nonsyn_mutations'], x=sample_data['s501y'], boxpoints=False))
fig.update_layout(yaxis_title='S Mutations', xaxis_title='B1117 Lineage',
                  template='plotly_white', autosize=False, height=850, width=800)
# fig.show()

In [ ]:
# world plot of variant prevalence

## County Level

In [230]:
from urllib.request import urlopen
import json

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [268]:
county_map = {x['properties']['NAME']+x['properties']['STATE']: x['id'] for x in counties['features']}

In [269]:
for c in county_map:
    if 'Orange' in c:
        print(c)
        print(county_map[c])

Orange12
12095
Orangeburg45
45075
Orange06
06059
Orange18
18117
Orange48
48361
Orange36
36071
Orange37
37135
Orange51
51137
Orange50
50017


In [328]:
us.loc[us['location']=='unk', 'idx'].unique().shape

(25407,)

In [261]:
us

Index(['idx', 'replacements', 'pos', 'gene', 'codon_num', 'ref_codon',
       'alt_codon', 'ref_aa', 'alt_aa', 'strain', 'virus', 'gisaid_epi_isl',
       'genbank_accession', 'date', 'region', 'country', 'division',
       'location', 'region_exposure', 'country_exposure', 'division_exposure',
       'segment', 'length', 'host', 'age', 'sex', 'Nextstrain_clade',
       'pangolin_lineage', 'GISAID_clade', 'originating_lab', 'submitting_lab',
       'authors', 'url', 'title', 'paper_url', 'date_submitted', 'month',
       'is_andersen', 'county_code'],
      dtype='object')

In [249]:
us['county_code'] = us['location'].apply(lambda x: county_map.get(x, -1))
us.loc[us['county_code']==-1, 'idx'].unique().shape

(38074,)

In [250]:
us['idx'].unique().shape

(50868,)

In [270]:
sorted(us['location'].unique())

['Adams County WA',
 'Adams County WI',
 'Alachua County',
 'Alameda County',
 'Albany County',
 'Allamakee County',
 'Allegheny County',
 'Anchorage',
 'Angelina County',
 'Ashland County',
 'Asotin County',
 'Assumption Parish',
 'Atascosa County',
 'Atlanta',
 'Avoyelles Parish',
 'Bailey County',
 'Bandera County',
 'Barron County',
 'Bee County',
 'Benton County',
 'Bergen County',
 'Berkeley County',
 'Bethany',
 'Bexar County',
 'Bienville Parish',
 'Bloomfield',
 'Boone County',
 'Bossier Parish',
 'Boston',
 'Bowie County',
 'Branford',
 'Brazos County',
 'Brewster County',
 'Bronx',
 'Brooklyn',
 'Brooklyn WI',
 'Broome County',
 'Brown County TX',
 'Brown County WI',
 'Buchanan County',
 'Buffalo County',
 'Burleson County',
 'Burlington County',
 'Caddo Parish',
 'Calhoun County',
 'Camden County',
 'Cameron County',
 'Campbellsport',
 'Cass County',
 'Cattaraugus County',
 'Cayuga County',
 'Chautauqua County',
 'Chelan County',
 'Chemung County',
 'Chenango County',
 'Chi

In [229]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})
df

,fips,unemp
0,01001,5.3
1,01003,5.4
2,01005,8.6
3,01007,6.6
4,01009,5.5
...,...,...
3214,72145,13.9
3215,72147,10.6
3216,72149,20.2
3217,72151,16.9
